In [2]:
import pandas as pd
from tqdm import tqdm

import os
# from utils import reduce_mem_usage
import numpy as np

import random
from random import sample

random.seed(2020)

In [3]:
df_train_click = pd.read_csv('input/train_click_log.csv')
df_test_click = pd.read_csv('input/testA_click_log.csv')

df_train_click.shape, df_test_click.shape

((1112623, 9), (518010, 9))

In [4]:
df_train_click.head()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,199999,160417,1507029570190,4,1,17,1,13,1
1,199999,5408,1507029571478,4,1,17,1,13,1
2,199999,50823,1507029601478,4,1,17,1,13,1
3,199998,157770,1507029532200,4,1,17,1,25,5
4,199998,96613,1507029671831,4,1,17,1,25,5


In [3]:
len(set(df_train_click.user_id))

200000

In [4]:
len(set(list(set(df_train_click['click_article_id']))+list(set(df_test_click['click_article_id']))))

35380

In [5]:
len(set(df_test_click.user_id))

50000

In [6]:
df_train_click.head()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,199999,160417,1507029570190,4,1,17,1,13,1
1,199999,5408,1507029571478,4,1,17,1,13,1
2,199999,50823,1507029601478,4,1,17,1,13,1
3,199998,157770,1507029532200,4,1,17,1,25,5
4,199998,96613,1507029671831,4,1,17,1,25,5


In [7]:
# # 训练集采样
# all_user_ids = df_train_click.user_id.unique()
# sample_nums = len(all_user_ids) // 2

# sample_user_ids = np.random.choice(all_user_ids, size=sample_nums, replace=False)
# df_train_click = df_train_click[df_train_click['user_id'].isin(sample_user_ids)]

# df_train_click.shape, df_test_click.shape

In [8]:
train_users = df_train_click['user_id'].values.tolist()
# 随机采样出一部分样本
val_users  = sample(train_users, 1000)
len(set(val_users))

998

In [9]:
click_list = []
train_query_list = []

groups = df_train_click.groupby(['user_id'])

In [10]:
# 训练集用户 抽出行为数据最后一条作为label
click_list = []
train_query_list = []

groups = df_train_click.groupby(['user_id'])
for user_id, g in tqdm(groups):
    if user_id in val_users:
        train_query = g.tail(1)
        train_query_list.append(train_query[['user_id', 'click_article_id']])

        train_click = g.head(g.shape[0] - 1)
        click_list.append(train_click)
    else:
        click_list.append(g)

df_train_click = pd.concat(click_list, sort=False)
#计算item cf
#得到你可能最喜欢的5个新闻
#如果他最终在你点击过最后的那篇文章里面，那就是1，否则为0 

df_train_query = pd.concat(train_query_list, sort=False)

print(df_train_click.shape, df_train_query.shape)

100%|████████████████████████████████████████████████████████████████████████| 200000/200000 [00:26<00:00, 7519.76it/s]


(1111625, 9) (998, 2)


In [19]:
df_train_click.head()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
1112619,0,30760,1508211672520,4,1,17,1,25,2
1112620,0,157507,1508211702520,4,1,17,1,25,2
1112601,1,289197,1508211316889,4,1,17,1,25,6
1112602,1,63746,1508211346889,4,1,17,1,25,6
1112599,2,36162,1508211438695,4,3,20,1,25,2


In [22]:
df_train_query.head()

,user_id,click_article_id
1109839,358,72330
1109090,459,36162
1107316,647,318172
1102349,1216,226563
1101241,1347,277107


In [25]:
df_train_click[df_train_click['user_id']==358].head()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
1109834,358,72336,1508204741793,4,1,17,1,25,5
1109835,358,72335,1508204792639,4,1,17,1,25,1
1109836,358,72334,1508204804397,4,1,17,1,25,1
1109837,358,72333,1508204828341,4,1,17,1,25,1
1109838,358,72329,1508204854568,4,1,17,1,25,1


In [24]:
df_train_query[df_train_query['user_id']==358].head()

,user_id,click_article_id
1109839,358,72330


In [9]:
df_train_click.shape, df_test_click.shape

((9113, 9), (518010, 9))

In [26]:
test_users = df_test_click['user_id'].unique()
test_query_list = []

for user in tqdm(test_users):
    test_query_list.append([user, -1])

df_test_query = pd.DataFrame(test_query_list, columns=[
                             'user_id', 'click_article_id'])

100%|███████████████████████████████████████████████████████████████████████| 50000/50000 [00:00<00:00, 2005519.80it/s]


In [13]:
df_test_query.head()

,user_id,click_article_id
0,249999,-1
1,249998,-1
2,249997,-1
3,249996,-1
4,249995,-1


In [27]:
df_train_query.shape, df_test_query.shape

((996, 2), (50000, 2))

In [28]:
df_query = pd.concat([df_train_query, df_test_query],
                     sort=False).reset_index(drop=True)
df_click = pd.concat([df_train_click, df_test_click],
                     sort=False).reset_index(drop=True)

In [29]:
df_query.head()

,user_id,click_article_id
0,358,72330
1,459,36162
2,647,318172
3,1216,226563
4,1347,277107


In [31]:
df_click = df_click.sort_values(
    ['user_id', 'click_timestamp']).reset_index(drop=True)

In [32]:
df_click.head()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,0,30760,1508211672520,4,1,17,1,25,2
1,0,157507,1508211702520,4,1,17,1,25,2
2,1,289197,1508211316889,4,1,17,1,25,6
3,1,63746,1508211346889,4,1,17,1,25,6
4,2,36162,1508211438695,4,3,20,1,25,2


In [33]:
# df_click = reduce_mem_usage(df_click)
# df_query = reduce_mem_usage(df_query)

In [34]:
os.makedirs('data', exist_ok=True)
df_click.to_pickle('data/click.pkl')
df_query.to_pickle('data/query.pkl')